In [428]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score
from sklearn.datasets import load_iris
from itertools import combinations

In [429]:
iris = datasets.load_iris()
X1=iris.data
y1=iris.target
data= pd.DataFrame(iris.data)
data.columns = iris.feature_names
data['class'] = iris.target

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [430]:
data = load_iris()

In [431]:
data['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [437]:


class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.01,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if self.print_every:
                if i%self.print_every == 0 or i == self.epoch-1:
                    print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy


In [438]:
def get_combinations(items,number):
    comb_list = []
    for c in combinations(items, number):
        c = list(c)
        c.sort()
        comb_list.append(c)
        
    return comb_list

In [439]:
def wrapper_helper(data,feature_names,target_name,train_size = 0.6,test_size = 0.4):
    dataset = data['data'].copy()
    target = data['target'].reshape(-1,1)
    
    final_data = np.hstack((dataset,target))
    np.random.shuffle(final_data)
    
    pd_data = pd.DataFrame(final_data)
    pd_data.columns= ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)','target']
    pd_data = pd_data[(pd_data['target']==0) | (pd_data['target']==1)]
    pd_data.tail(2)
    

    col_names = np.concatenate((feature_names,target_name),axis=None)
    final_data = pd_data[col_names].values

    train_data = final_data[:int(len(final_data)*train_size)]
    test_data = final_data[int(len(final_data)*train_size):]
    
    

    train_data.shape,test_data.shape 
    
    X_train = train_data[:,:-1]
    y_train = train_data[:,-1:]


    X_test = test_data[:,:-1]
    y_test = test_data[:,-1:]


    return X_train,y_train,X_test,y_test

In [440]:
def wrapper_method(data,names,k=2):
    classes = get_combinations(names,k)
    wrapper_output = {}
    
    for i in classes:
        X_train,y_train,X_test,y_test = wrapper_helper(data,i,['target'])
        
        logistic = logisticregression(X_train,y_train,epoch=10,print_every=None)
        logistic.train()
        
        result = logistic.evaluate(X_test,y_test)
        wrapper_output[str(i)] = result
            
    return wrapper_output

In [442]:
wrapper_method(data,data['feature_names'],2)

{"['sepal length (cm)', 'sepal width (cm)']": 0.575,
 "['petal length (cm)', 'sepal length (cm)']": 0.9,
 "['petal width (cm)', 'sepal length (cm)']": 0.45,
 "['petal length (cm)', 'sepal width (cm)']": 1.0,
 "['petal width (cm)', 'sepal width (cm)']": 1.0,
 "['petal length (cm)', 'petal width (cm)']": 0.95}